In [1]:
!pip install yfinance
!pip install yahooquery
!pip install tvdatafeed
!pip install tensorflow
from tvDatafeed import TvDatafeed, Interval
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model
import os
import shutil
from datetime import datetime
import glob

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 714 kB/s 
     |████████████████████████████████| 6.4 MB 41.8 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |██████████

In [89]:
def work_with_dir():
  if os.path.exists("/content/data/"):
    shutil.rmtree("/content/data/", ignore_errors=True)
    print("Data Folder Removed")
    os.mkdir("/content/data/")
  if not os.path.exists("/content/data/"):
    os.mkdir("/content/data/")
  if not os.path.exists("/content/extracted/"):
    os.mkdir("/content/extracted/")
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["suggestion"], axis=1)
  y = data[["suggestion"]]
  y = pd.get_dummies(y)
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.2,random_state=99)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def download_data(symbols,periodd,intervall):
  indexx = 100
  work_with_dir()
  for symbol in symbols:
    if ((symbols.index(symbol)+1) % 100 == 0):
      print(f" -- {indexx}",end="")
      indexx = indexx + 100
    try:
        data = yf.download(symbol, period=periodd,interval=intervall, progress=False,show_errors=False)
        if data.empty:
           pass
        else:
             data.to_csv(f"/content/data/{symbol}.csv")         
    except:
       print("Error!")
  print(" ")
def each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
     data = []
     unattached_dfs = []
     files = list(files)
     for file in files:
        #print(f"{files.index(file)+1+index}",end=" ")
        if (files.index(file)+index+1) % 10 == 0:
           print(f"{files.index(file)+1+index}",end=" ")
           if (files.index(file)+index+1) % 400 == 0:
             print(" ")

        address = f"/content/data/{file}"
        try:
            unattached_dfs.append(process(pd.read_csv(address),min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that))
        except:
          pass
     nud = []
     for i in unattached_dfs:
       if i.size != 0:
         nud.append(i)
     data = np.concatenate(nud)
     data = pd.DataFrame(data, columns=clmns)
     right_now = datetime.now().strftime("%H%M%S%f")
     data.to_csv(f"/content/extracted/{now}/{right_now}.csv")  
def extract_data(pieces,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
  print(f"Files In Data : {len(os.listdir('/content/data/'))}")
  pd.options.mode.chained_assignment = None
  files = os.listdir("/content/data/")
  new_files = np.array_split(files, pieces)
  print("Processing File:")
  now = datetime.now().strftime("%H%M%S")
  os.mkdir(f"/content/extracted/{now}/")
  index = 0 
  for files in new_files:
     each_file_proc(files,now,index,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     index = index + len(files)
  print(" ")
  return now
def delete_all_csv(now):
   path = f'/content/extracted/{now}/*.csv'
   files = glob.glob(path)
   for file in files:
       os.remove(file)
def make_df(now):
   path = f'/content/extracted/{now}/*.parquet'
   files = glob.glob(path)
   #data = pd.DataFrame()
   data = pd.DataFrame()
   for adr in files:
     data =pd.concat([data,pd.read_parquet(adr)])
   if "Unnamed: 0" in data:
     data.drop("Unnamed: 0",axis=1,inplace=True)
   print(data.shape)
   xTrain,xTest,yTrain,yTest = spliting(data)
   data.to_parquet(f'/content/extracted/{now}/data.parquet')
   delete_all_csv(now)
   data = []
   return xTrain,xTest,yTrain,yTest
def to_par(now,howmanyfiles): 
    files = os.listdir(f"/content/extracted/{now}/")
    addresses = []
    for file in files:
      addresses.append(f"/content/extracted/{now}/{file}")
    new_adr = np.array_split(addresses,howmanyfiles)
    for adrs in new_adr:
      datas = []
      for adr in adrs:
        datas.append(pd.read_csv(adr))
      rnow = datetime.now().strftime("%H%M%S%f")
      datas = pd.concat(datas)
      datas.to_parquet(f"/content/extracted/{now}/part_{rnow}.parquet")   
def scaler(row,min_range,max_range):
    scaler = MinMaxScaler(feature_range=(min_range, max_range))
    row = scaler.fit_transform(row)
    return row
def make_y(data, i,how_many_future_candles,how_many_past_candles):
  mean_of_next = []
  for ix in range(i,i+how_many_future_candles):
    mean_of_next.append(data[ix][0]+data[ix][3])
  mean_of_next = [element / 2 for element in mean_of_next]
  mean_of_next = sum(mean_of_next)/how_many_future_candles
  mean_of_previous = []
  for ix in range(i-how_many_past_candles,i):
    mean_of_previous.append(data[ix][0]+data[ix][3])
  mean_of_previous = [element / 2 for element in mean_of_previous]
  mean_of_previous = sum(mean_of_previous)/how_many_past_candles
  if mean_of_next > mean_of_previous:
    return "buy"
  else:
    return "sell"
def process(data,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that):
    data = data.dropna()
    row = []
    if len(data.columns) == 7:
      data = data.iloc[:, 1:]
    data = np.array(data)
    for i in range(each_row_past, data.shape[0]-20):
        opens = []
        highs = []
        lows = []
        closes = []           
        for ix in range(each_row_past, 0, interval_for_that):
            opens.append([(data[i-ix][0]+data[i-ix+1][0]+data[i-ix+2][0]) /3 - (data[i-ix+3][0]+data[i-ix+4][0]+data[i-ix+5][0])/3])
            closes.append([(data[i-ix][3]+data[i-ix+1][3]+data[i-ix+2][3]) /3 - (data[i-ix+3][3]+data[i-ix+4][3]+data[i-ix+5][3])/3])
        sugg = make_y(data, i,how_many_future_candles,how_many_past_candles)
        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1),min_range,max_range)
        arr = np.append(arr, sugg)
        row.append(arr)
    return np.array(row)
def start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet):
     folder_name = extract_data(into_how_many_csv,min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that)
     to_par(folder_name,csv_to_parquet)
     xTrain,xTest,yTrain,yTest = make_df(folder_name)
     return xTrain,xTest,yTrain,yTest  

In [20]:
symbols = get_crypto_syms()
print(f"Symbols : {len(symbols)}")
download_data(symbols,"max","1wk")

Symbols : 1500
Data Folder Removed
 -- 100 -- 200 -- 300 -- 400 -- 500 -- 600 -- 700 -- 800 -- 900 -- 1000 -- 1100 -- 1200 -- 1300 -- 1400 -- 1500 


In [104]:
clmns = list(range(1, 51))
clmns.append("suggestion")

min_range,max_range = -10,10
how_many_future_candles = 3
how_many_past_candles = 3
each_row_past,interval_for_that = 150,-6

into_how_many_csv = 50
csv_to_parquet = 10

xTrain,xTest,yTrain,yTest = start(min_range,max_range,how_many_future_candles,how_many_past_candles,each_row_past,interval_for_that,into_how_many_csv,csv_to_parquet)
print(yTrain.value_counts())
print(yTest.value_counts())

Files In Data : 1499
Processing File:
10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400  
410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800  
810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200  
1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490  
(53270, 51)
(42616, 50) (42616, 2)
(10654, 50) (10654, 2)
suggestion_buy  suggestion_sell
1               0                  21874
0               1                  20742
dtype: int64
suggestion_buy  suggestion_sell
1               0                  5556
0               1                  5098
dtype: int64


In [105]:
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(2, activation='sigmoid'))

opt = tf.keras.optimizers.Adamax()

model.compile(optimizer=opt, loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_91 (Dense)            (None, 512)               26112     
                                                                 
 dense_92 (Dense)            (None, 512)               262656    
                                                                 
 dense_93 (Dense)            (None, 512)               262656    
                                                                 
 dense_94 (Dense)            (None, 512)               262656    
                                                                 
 dense_95 (Dense)            (None, 512)               262656    
                                                                 
 dense_96 (Dense)            (None, 2)                 1026      
                                                                 
Total params: 1,077,762
Trainable params: 1,077,762
N

In [106]:
model.fit(xTrain,yTrain,epochs=30,batch_size=32,validation_data=(xTest,yTest))
acr = str(round(model.evaluate(xTest,yTest)[1],4)).replace("0.","")
model.save(f"MCG136_{acr}.h5")

Epoch 1/30
1332/1332 [==============================] - 20s 14ms/step - loss: 0.4348 - accuracy: 0.7966 - val_loss: 0.3701 - val_accuracy: 0.8368
Epoch 2/30
1332/1332 [==============================] - 19s 14ms/step - loss: 0.2984 - accuracy: 0.8714 - val_loss: 0.2597 - val_accuracy: 0.8886
Epoch 3/30
1332/1332 [==============================] - 19s 14ms/step - loss: 0.2127 - accuracy: 0.9113 - val_loss: 0.2234 - val_accuracy: 0.9109
Epoch 4/30
1332/1332 [==============================] - 19s 14ms/step - loss: 0.1638 - accuracy: 0.9343 - val_loss: 0.1712 - val_accuracy: 0.9362
Epoch 5/30
1332/1332 [==============================] - 19s 14ms/step - loss: 0.1310 - accuracy: 0.9486 - val_loss: 0.1669 - val_accuracy: 0.9411
Epoch 6/30
1332/1332 [==============================] - 19s 14ms/step - loss: 0.1152 - accuracy: 0.9541 - val_loss: 0.1624 - val_accuracy: 0.9361
Epoch 7/30
1332/1332 [==============================] - 19s 14ms/step - loss: 0.1004 - accuracy: 0.9604 - val_loss: 0.1343 -

In [86]:
def process_for_prediction(data,index,min_range,max_range,each_row_past,interval_for_that):
        i = index
        if "symbol" in data.columns:
              data.drop("symbol",axis=1,inplace=True) 
        if "datetime" in data.columns:
              data.drop("datetime",axis=1,inplace=True)
        if "Adj Close" in data.columns:
              data.drop("Adj Close",axis=1,inplace=True)

        data = np.array(data)

        opens = []
        highs = []
        lows = []
        closes = []
        for ix in range(each_row_past, 0, interval_for_that):
            opens.append([(data[i-ix][0]+data[i-ix+1][0]+data[i-ix+2][0]) /3 - (data[i-ix+3][0]+data[i-ix+4][0]+data[i-ix+5][0])/3])
            closes.append([(data[i-ix][3]+data[i-ix+1][3]+data[i-ix+2][3]) /3 - (data[i-ix+3][3]+data[i-ix+4][3]+data[i-ix+5][3])/3])

        arr = np.concatenate((np.array(opens).flatten(), np.array(highs).flatten()
        , np.array(lows).flatten(), np.array(closes).flatten()), axis=0).reshape(-1, 1)
        arr = scaler(arr.reshape(-1, 1),min_range,max_range)
        return arr,0
def make_prediction_for_yf(symbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that):
    raw_data, mean_of_previous = process_for_prediction(
        yf.download(symbol, period=period, interval=timeframe), index,min_range,max_range,each_row_past,interval_for_that)
    return f"""YF   : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction_for_tv(symbol,exchange,timeframe,tindex,min_range,max_range,each_row_past,interval_for_that):
    tv = TvDatafeed()
    raw_data, mean_of_previous = process_for_prediction(
       tv.get_hist(symbol=symbol, exchange=exchange, interval=timeframe, n_bars=5000), tindex,min_range,max_range,each_row_past,interval_for_that)
    return f"""TVB  : {model.predict(np.array(raw_data).reshape(1,-1)).tolist()}
MP   : {mean_of_previous}
"""
def make_prediction(ysymbol,period,timeframe,tsymbol,texchange,ttimeframe,tindex,index):
  print(make_prediction_for_yf(ysymbol,period,timeframe,index,min_range,max_range,each_row_past,interval_for_that))
  print(make_prediction_for_tv(tsymbol,texchange,ttimeframe,tindex,min_range,max_range,each_row_past,interval_for_that))

In [87]:
make_prediction("btc-usd","max","1d","btcusd","bitstamp",Interval.in_daily,-1,-1)

[*********************100%***********************]  1 of 1 completed


YF   : [[2.373982260772678e-11, 1.0]]
MP   : 0

TVB  : [[5.934534114127032e-11, 1.0]]
MP   : 0



In [ ]:
yf.download("btc-usd",period="max",interval="1mo")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')